In [10]:
import os
import sys
from importlib import reload

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.insert(0,module_path)


import pandas as pd
import numpy as np
from skmultilearn.problem_transform import ClassifierChain
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_fscore_support,roc_auc_score
from skmultilearn.model_selection import IterativeStratification
import sklearn.preprocessing as pp
import altair as alt
alt.renderers.enable('notebook')

from networks.functionsDENet import loadPickle
from stages_DE.stages_library import PHENOTYPES

In [2]:
pathClassification = '/home/karin/Documents/timeTrajectories/data/stages/classification'

In [5]:
X_train, Y_train, X_test, Y_test = loadPickle(pathClassification + 'train_test.pkl')
order=PHENOTYPES.copy()
order.remove('tag_spore')

In [8]:
# split = LeaveOneOut()
prfs_all=pd.DataFrame()
rac_all=pd.DataFrame()
feats_all=pd.DataFrame()
split = IterativeStratification(n_splits=5, order=1)
fold=0
for train_index, test_index in split.split(X_train, Y_train):
    fold += 1
    print(fold)
    X_train_fold, X_test_fold = pp.minmax_scale(X_train[train_index]), pp.minmax_scale(X_train[test_index])
    Y_train_fold, Y_test_fold = pp.minmax_scale(Y_train[train_index]), pp.minmax_scale(Y_train[test_index])
    for c in [0.9,0.7,0.5,0.3,0.2,0.1]:
        print(c)
        # Order already ensured when selecting Y columns
        classifier = ClassifierChain(
            classifier=LogisticRegression(penalty='l1', n_jobs=3, C=c, solver='saga', max_iter=4000)).fit(X_train_fold,
                                                                                                         Y_train_fold)

        Y_predict_fold = classifier.predict(X_test_fold)
        Y_p_fold = classifier.predict_proba(X_test_fold)
        prfs=pd.DataFrame(precision_recall_fscore_support(Y_test_fold, Y_predict_fold),index=['precision','recall','F_score','support']).T
        prfs['Stage']=order
        prfs['C']=[c]*prfs.shape[0]
        prfs_all=prfs_all.append(prfs)
        prfs=list(precision_recall_fscore_support(Y_test_fold, Y_predict_fold, average='micro'))
        prfs.extend(['all',c])
        prfs=dict(zip(['precision','recall','F_score','support','Stage',"C"],prfs))
        prfs_all = prfs_all.append( prfs,ignore_index=True)
        rac=dict(zip(['roc_auc','C'],[ roc_auc_score(Y_test_fold, Y_p_fold.toarray(), average='micro'),c]))
        rac_all=rac_all.append(rac,ignore_index=True)

        feats_combined = set()
        for i in range(len(order)):
            cl = classifier.classifiers_[i]
            feats_stage = set(pd.Series(range(X_train.shape[1]))[(cl.coef_ != 0).flatten()[:X_train.shape[1]]].index)
            feats_combined = feats_combined | feats_stage
            #print(len(feats_stage), len(feats_combined))
            feats=dict(zip(['N_features','C','Stage'],[len(feats_stage),c,order[i]]))
            feats_all=feats_all.append(feats,ignore_index=True)
        feats= dict(zip(['N_features', 'C', 'Stage'],[len(feats_combined), c, 'all']))
        feats_all = feats_all.append(feats,ignore_index=True)

1
0.9


/home/karin/Documents/git/orange3venv/lib/python3.6/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/karin/Documents/git/orange3venv/lib/python3.6/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/karin/Documents/git/orange3venv/lib/python3.6/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/karin/Documents/git/orange3venv/lib/python3.6/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/karin/Documents/git/orange3venv/lib/python3.6/site

0.7


/home/karin/Documents/git/orange3venv/lib/python3.6/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/karin/Documents/git/orange3venv/lib/python3.6/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/karin/Documents/git/orange3venv/lib/python3.6/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/karin/Documents/git/orange3venv/lib/python3.6/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/karin/Documents/git/orange3venv/lib/python3.6/site

0.5


...

/home/karin/Documents/git/orange3venv/lib/python3.6/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/karin/Documents/git/orange3venv/lib/python3.6/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/karin/Documents/git/orange3venv/lib/python3.6/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/karin/Documents/git/orange3venv/lib/python3.6/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/karin/Documents/git/orange3venv/lib/python3.6/site

0.2


/home/karin/Documents/git/orange3venv/lib/python3.6/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/karin/Documents/git/orange3venv/lib/python3.6/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/karin/Documents/git/orange3venv/lib/python3.6/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/karin/Documents/git/orange3venv/lib/python3.6/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/karin/Documents/git/orange3venv/lib/python3.6/site

0.1


/home/karin/Documents/git/orange3venv/lib/python3.6/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/karin/Documents/git/orange3venv/lib/python3.6/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/karin/Documents/git/orange3venv/lib/python3.6/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/karin/Documents/git/orange3venv/lib/python3.6/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/karin/Documents/git/orange3venv/lib/python3.6/site

In [1]:
alt.Chart(prfs_all).mark_circle(size=20).encode(x='C',y='precision',color='Stage').transform_calculate(
    jitter='sqrt(-2*log(random()))*cos(2*PI*random())'
)

NameError: name 'alt' is not defined